In [1]:
pip install openpyxl requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import json
import requests
from bs4 import BeautifulSoup
import time

url = 'https://www.chambredesrepresentants.ma/fr/%D9%85%D8%B1%D8%A7%D9%82%D8%A8%D8%A9-%D8%A7%D9%84%D8%B9%D9%85%D9%84-%D8%A7%D9%84%D8%AD%D9%83%D9%88%D9%85%D9%8A/%D8%A7%D9%84%D8%A3%D8%B3%D9%80%D8%A6%D9%84%D8%A9-%D8%A7%D9%84%D8%B4%D9%81%D9%88%D9%8A%D8%A9-%D8%A3%D8%B1%D8%B4%D9%8A%D9%81'

data = []
try:
    while url:
        response = None
        while response is None:
            try:
                response = requests.get(url)
            except requests.exceptions.RequestException as e:
                print(f"Erreur de connexion : {e}")
                time.sleep(5)  # Attendre 5 secondes avant de réessayer
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            for x in soup.find_all(class_="q-b3-item q-b3i-red"):
                minister_txt = x.select_one('span:-soup-contains(" Le ministère compétent   ")')
                if minister_txt:
                    minister_txt = minister_txt.parent.text.replace(" Le ministère compétent   ", "").strip()
                else:
                    minister_txt = "none"
                    
                date_txt = x.select_one('span:-soup-contains("Date : ")')
                if date_txt:
                    date_txt = date_txt.parent.text.replace("Date :", "").strip()
                else:
                    date_txt = "none"
                            
                question_link = x.find('a', href=True)
                if question_link:
                    question_link = question_link['href']
                    Url2 = f"https://www.chambredesrepresentants.ma{question_link}"
                    response2 = requests.get(Url2)
                    if response2.status_code == 200:
                        soup2 = BeautifulSoup(response2.text, 'html.parser')
                        
                        qnum_txt = soup2.select_one('span:-soup-contains("رقم السؤال :")')
                        if qnum_txt:
                            qnum_txt = qnum_txt.parent.text.replace("رقم السؤال :", "").strip()
                        else:
                            qnum_txt = "none"
                        
                        obj = soup2.select_one('span:-soup-contains("Objet :")')
                        if obj:
                            obj = obj.parent.text.replace("Objet :", "").strip()
                        else:
                            obj = "none"
                        
                        response_date_txt = soup2.select_one('span:-soup-contains("Date réponse :")')
                        if response_date_txt:
                            response_date_txt = response_date_txt.parent.text.replace("Date réponse :", "").strip()
                        else:
                            response_date_txt = "none"
                        
                        question_txt = soup2.select_one('span:-soup-contains("Question : ")')
                        if question_txt:
                            question_txt = question_txt.parent.text.replace("Question :", "").strip()
                        else:
                            question_txt = "none"
                        
                        nom_questionneur = soup2.select_one('.q-name a')
                        if nom_questionneur:
                            nom_questionneur = nom_questionneur.text.strip()
                        else:
                            nom_questionneur = "none"
                        
                        link_questioner = soup2.select_one('.q-name').a['href']
                        Url3 = f"https://www.chambredesrepresentants.ma{link_questioner}"
                        response3 = requests.get(Url3)
                        if response3.status_code == 200:
                            soup3 = BeautifulSoup(response3.text, 'html.parser')
                            div = soup3.find('div', class_="mhr-b1-info-l").find_next('div')
                            div2 = div.find_next('div')
                            groupe = div2.find('span', class_="mht-t").find_next('span')
                            if groupe:
                                groupe = groupe.text.strip()
                            else:
                                groupe = "none"
                        
                        # Créer un dictionnaire pour les données extraites
                        entry = {
                            "minister_txt": minister_txt,
                            "qnum_txt": qnum_txt,
                            "obj": obj,
                            "response_date_txt": response_date_txt,
                            "date_txt": date_txt,
                            "question_txt": question_txt,
                            "nom_questionneur": nom_questionneur,
                            "groupe": groupe
                        }
                        
                        data.append(entry)

            # Find the <ul> element with class "pagination"
            pagination_ul = soup.find('ul', class_='pagination')
            # Check if there is a "next" link
            next_link = pagination_ul.find('li', class_='next')
            if next_link:
                link = next_link.find('a')
                if link:
                    url1 = link['href']
                    url = f"https://www.chambredesrepresentants.ma{url1}"
                    print(f"Next Link Href: {url}")
                    print(data)
            else:
                url = None

except Exception as e:
    print(f"Une exception s'est produite : {e}")

finally:
    # Enregistrer les résultats dans un fichier JSON
    with open('resultats_orale.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

Next Link Href: https://www.chambredesrepresentants.ma/fr/%D9%85%D8%B1%D8%A7%D9%82%D8%A8%D8%A9-%D8%A7%D9%84%D8%B9%D9%85%D9%84-%D8%A7%D9%84%D8%AD%D9%83%D9%88%D9%85%D9%8A/%D8%A7%D9%84%D8%A3%D8%B3%D9%80%D8%A6%D9%84%D8%A9-%D8%A7%D9%84%D8%B4%D9%81%D9%88%D9%8A%D8%A9-%D8%A3%D8%B1%D8%B4%D9%8A%D9%81?page=1
[{'minister_txt': "Secrétariat d'Etat chargé de la formation professionnelle.", 'qnum_txt': '258', 'obj': 'وضعية بعض مستشفيات المملكة', 'response_date_txt': 'pas de réponse', 'date_txt': '05/11/2021', 'question_txt': 'تعاني العديد من مستشفيات المملكة نقصا كبيرا في التجهيزات والمعدات الطبية ومن خصاص حاد على مستوى الموارد البشرية ، و مما زاد في تعرية هذا الواقع ماخلفته أزمة كورونا من انعكاسات سلبية على جودة الخدمات المقدمة للمرضى .\nلأجل ذلك وفي إطار سياسة تأهيل المستشفيات لكي تقوم بواجباتها الصحية لصالح المواطنات والمواطنين ، يظل المستشفى الإقليمي بإنزكان في حاجة ماسة إلى إعادة هيكلته إداريا وبشريا ولوجيستيكيا  هذا في الوقت الذي يظل جناحه  المختص في الولادة لا يتوفر إلا على سرير  واحد لكل ثلاث

KeyboardInterrupt: 